In [1]:
# import src.image_processing as imgproc

In [3]:
import os
import subprocess
from re import sub
from glob import glob
from tqdm                   import tqdm
import multiprocessing      as mp
from functools              import partial
from src.utils import execute_R
from pyminc.volumes.factory import volumeFromFile
import numpy as np
from warnings import warn
import pandas as pd

def gunzip_file(gzfile, keep = True, outdir = None):
    
    """
    Unzip a compressed file.
    
    Arguments
    ---------
    gzfile: str
        Path to the file to unzip.
    keep: bool
        Option to keep the compressed file after extraction.
    outdir: str
        Path to the output directory. If None, the file will
        be unzipped in its native directory.
        
    Returns
    -------
    outfile: str
        Path to the unzipped file.
    """
    
    os.system('gunzip -f -k {}'.format(gzfile))
    outfile = sub(r'.gz', '', gzfile)
    if not keep:
        os.system('rm {}'.format(gzfile))
    if outdir is not None:
        outdir = os.path.join(outdir, '')
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        os.system('mv {} {}'.format(outfile, outdir))
        outfile = os.path.join(outdir, os.path.basename(outfile))

    return outfile


def gunzip_files(infiles, keep = True, outdir = None, parallel = False, nproc = None):
    
    """
    Unzip a set of compressed files.
    
    Arguments
    ---------
    infiles: list
        List of paths to files to unzip.
    keep: bool
        Option to keep the compressed files after extraction.
    outdir: None
        Path to the output directory. If None, the files will
        be unzipped in their native directories.
    parallel: bool
        Option to run in parallel.
    nprpc: int
        Number of processors to use in parallel.
        
    Returns
    -------
    outfiles: list
        List of paths to the unzipped files.
    """
    
    gunzip_file_partial = partial(gunzip_file, 
                                  keep = keep,
                                  outdir = outdir)
    if parallel:
        if nproc is None:
            raise ValueError("Set the nproc argument to specify the number of processors to use")
        pool = mp.Pool(nproc)
        outfiles = []
        for outfile in tqdm(pool.imap(gunzip_file_partial, infiles), total = len(infiles)):
            outfiles.append(outfile)
        pool.close()
        pool.join()
    else:
        outfiles = list(map(gunzip_file_partial, tqdm(infiles)))
        
    return outfiles


def nii_to_mnc(infile, keep = True, outdir = None):
    
    """
    Convert a NIFTY file to MINC format.
    
    Arguments
    ---------
    infile: str
        Path to the NIFTY file to convert.
    outdir: str
        Path to the output directory. If None, the output
        MINC file will be stored in the native directory.
    keep: bool
        Option to keep the input file. 
        
    Returns
    -------
    outfile: str
        Path to the converted MINC file.
    """
    
    os.system('nii2mnc -quiet -clobber {}'.format(infile))
    outfile = sub(r'.nii', '.mnc', infile)
    if not keep:
        os.system('rm {}'.format(infile))
    if outdir is not None:
        outdir = os.path.join(outdir, '')
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        os.system('mv {} {}'.format(outfile, outdir))
        outfile = os.path.join(outdir, os.path.basename(outfile))
        
    return outfile


def mnc_to_nii(infile, keep = True, outdir = None):
    
    """
    Convert a MINC file to NIFTY format.
    
    Arguments
    ---------
    infile: str
        Path to the MINC file to convert.
    outdir: str
        Path to the output directory. If None, the output
        MINC file will be stored in the native directory.
    keep: bool
        Option to keep the input file. 
        
    Returns
    -------
    outfile: str
        Path to the converted NIFTY file.
    """
    
    os.system('mnc2nii -quiet {}'.format(infile))
    outfile = sub(r'.mnc', '.nii', infile)
    if not keep:
        os.system('rm {}'.format(infile))
    if outdir is not None:
        outdir = os.path.join(outdir, '')
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        os.system('mv {} {}'.format(outfile, outdir))
        outfile = os.path.join(outdir, os.path.basename(outfile))
        
    return outfile
    
    
def convert_images(infiles, input_format = 'nifty', output_format = 'minc', keep = True, outdir = None, parallel = False, nproc = None):
    
    """
    Convert a set of images from NIFTY to MINC or vice versa.
    
    Arguments
    ---------
    infiles: list
        List of paths to images to convert.
    input_format: str
        One of {'nifty', 'minc'} indicating the input format.
    output_format: str
        One of {'nifty', 'minc'} indicating the output format. 
        Must be different from the input format.
    keep: bool
        Option to keep the input file. 
    outdir: str
        Path to the output directory. If None, the converted
        images will be stored in the native directory.
    parallel: bool
        Option to run in parallel.
    nprpc: int
        Number of processors to use in parallel.
        
    Returns
    -------
    outfiles: list
        List of paths to the converted images.
    """
        
    if (input_format == 'nifty') & (output_format == 'minc'):
        converter = partial(nii_to_mnc, keep = keep, outdir = outdir)
    elif (input_format == 'minc') & (output_format == 'nifty'):
        converter = partial(mnc_to_nii, keep = keep, outdir = outdir)
    else:
        raise ValueError 
        
    if parallel:
        if nproc is None:
            raise ValueError("Set the nproc argument to specify the number of processors to use")
        pool = mp.Pool(nproc)
        outfiles = []
        for outfile in tqdm(pool.imap(converter, infiles), total = len(infiles)):
            outfiles.append(outfile)
        pool.close()
        pool.join()
    else:
        outfiles = list(map(converter, tqdm(infiles)))
        
    return outfiles
    
    
def calculate_human_effect_sizes(demographics, imgdir, maskfile, outdir, ncontrols = 10, threshold = 10, dataset = 1, parallel = False, nproc = None):

    """
    Calculate human effect size images.
    
    Arguments
    ---------
    demographics: str
        Path to the CSV file containing the human demographics data.
    imgdir: str
        Path to the directory containing the MINC images to use to compute the effect sizes.
    maskfile: str
        Path to the mask MINC file for the images.
    outdir: str
        Path to the directory in which to save the effect size MINC images.
    ncontrols: int
        Number of propensity-matched controls to use when computing the effect sizes.
    threshold: int
    dataset: int
        A numeric flag indicating which data to use. Set to 1 for all data. Set to 2 for POND and SickKids. Set to 3 for POND only.
    parallel: bool
        Option to run in parallel.
    nproc: int
        Number of processors to use in parallel.
    
    Returns
    -------
    outfiles: list
        List of paths to the effect size images.
    """

    script = 'calculate_human_effect_sizes.R'
    if parallel:
        if nproc is None:
            raise ValueError("Set the nproc argument to specify the number of processors to use")
    else: 
        nproc = 1
    parallel = 'true' if parallel else 'false'
    script_args = {'demographics':demographics,
                   'imgdir':imgdir,
                   'maskfile':maskfile,
                   'outdir':outdir,
                   'ncontrols':ncontrols,
                   'threshold':threshold,
                   'dataset':dataset,
                   'parallel':parallel,
                   'nproc':nproc}
    execute_R(script = script, args = script_args)
    outfiles = os.listdir(outdir)
    return outfiles


def resample_image(infile, isostep, outdir = None):
    
    """
    Resample a MINC image
    
    Arguments
    ---------
    infile: str
        Path to the image file to resample.
    isostep: float
        Isotropic dimension of voxels in the resampled image (millimeters).
    outdir: str
        Path to the directory in which to save the resampled image. If None, resampled image will be written to the directory of the input file.
    
    
    Returns
    -------
    outfile: str
        Path to the resampled image. 
    """
    
    outfile = sub(r'.mnc', '_resampled_{}.mnc'.format(isostep), infile)
    if outdir is not None:
        outdir = os.path.join(outdir, '')
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        outfile = os.path.basename(outfile)
        outfile = os.path.join(outdir, outfile)
    cmd_autocrop = ('autocrop -quiet -clobber -isostep {} {} {}'
                    .format(isostep, infile, outfile))
    os.system(cmd_autocrop)

    return outfile


def resample_images(infiles, isostep, outdir = None, parallel = False, nproc = None):
    
    """
    Resample a set of MINC images.
     
    Arguments
    ---------
    infiles: list
        List of paths to images to resample.
    outdir: str
        Path to the output directory. If None, the resampled 
        images will be stored in the native directory.
    parallel: bool
        Option to run in parallel.
    nproc: int
        Number of processors to use in parallel.
        
    Returns
    -------
    outfiles: list
        List of paths to the resampled images.
    """

    resampler = partial(resample_image,
                        isostep = isostep,
                        outdir = outdir)
    
    if parallel:
        if nproc is None:
            raise ValueError("Set the nproc argument to specify the number of processors to use in parallel.")
        pool = mp.Pool(nproc)
        outfiles = []
        for outfile in tqdm(pool.imap(resampler, infiles), total = len(infiles)):
            outfiles.append(outfile)
        pool.close()
        pool.join()
    else:
        outfiles = list(map(resampler, tqdm(infiles)))
        
    return outfiles
        

def import_image(img, mask = None, flatten = True):
    
    """
    Import a MINC image.
    
    Arguments
    ---------
    img: str
        Path to the MINC image to import.
    mask: str
        Optional path to the MINC mask. 
    flatten: bool
        Option to flatten image into a 1-dimensional array. 
        If True and a mask is provided, only the voxels in the mask will be returned.
    
    Returns
    -------
    img: numpy.ndarray
        A NumPy array containing the (masked) image.
    """
    
    img_vol = volumeFromFile(img)
    img_dims = img_vol.getSizes()
    img_seps = img_vol.getSeparations()
    img = np.array(img_vol.data)
    img_vol.closeVolume()

    if flatten:
        img = img.flatten()

    if mask is not None:

        mask_vol = volumeFromFile(mask)
        mask_dims = mask_vol.getSizes()
        mask_seps = mask_vol.getSeparations()
        mask = np.array(mask_vol.data)
        mask_vol.closeVolume()

        if mask_seps != img_seps:
            raise Exception("Input image and mask have different resolutions.")
        if mask_dims != img_dims:
            raise Exception("Input image and mask have different dimensions.")

        if flatten:
            mask = mask.flatten()
            img = img[mask == 1]
        else:
            img[mask == 0] = 0
    
    return img


def import_images(infiles, mask = None, output_format = 'list', flatten = True, parallel = False, nproc = None):

    """
    Import a set of MINC images.
    
    Arguments
    ---------
    infiles: list
        List of paths to images to import.
    mask: str
        Optional path to the MINC mask. 
    output_format: str
        One of {'list', 'numpy', 'pandas'} indicating what format to return.
    flatten: bool
        Option to flatten images into a 1-dimensional array. 
        If True and a mask is provided, only the voxels in the mask will be returned.
        If False and output_format is not 'list', images will be flattened regardless.
    parallel: bool
        Option to run in parallel.
    nproc: int
        Number of processors to use in parallel.
    
    Returns
    -------
    imgs
        A list, NumPy array, or Pandas DataFrame containing the (masked) images.
    """

    format_opts = ['list', 'numpy', 'pandas']
    format_test = sum([output_format == opt for opt in format_opts])
    format_err = ("Argument output_format must be one of {}: {}"
                   .format(format_opts, output_format))
    if format_test != 1:
        raise ValueError(format_err)

    if not flatten:
        if output_format != 'list':
            msg_warn = ("flatten = False is only valid when output_format = 'list'. "
                        "Proceeding with flattened images.")
            warn(msg_warn)
            flatten = True

    importer = partial(import_image,
                       mask = mask,
                       flatten = flatten)

    if parallel:
        if nproc is None:
            raise ValueError("Set the nproc argument to specify the number of processors to use in parallel.")
        pool = mp.Pool(nproc)
        imgs = []
        for img in tqdm(pool.imap(importer, infiles), total = len(infiles)):
            imgs.append(img)
        pool.close()
        pool.join()
    else:
        imgs = list(map(importer, tqdm(infiles)))

    imgsize_test = [len(img) for img in imgs]
    imgsize_err = "Images provided contain different numbers of voxels."
    if len(set(imgsize_test)) != 1:
        raise Exception(imgsize_err)

    if output_format == 'numpy':
        return np.asarray(imgs)
    elif output_format == 'pandas':
        return pd.DataFrame(np.asarray(imgs))
    else: 
        return imgs

In [4]:
def execute_R(script, args):
    
    """
    Execute an R script.
    
    Arguments
    ---------
    script: str
        String containing the name of the R script to execute.
    args: dict
        Dictionary of key-value pairs containing command line arguments to pass to the script
    
    Returns
    -------
    None
    """
    
    args = [['--'+str(key), str(val)] for key, val in args.items()]
    args = sum(args, [])
    cmd = ['Rscript']+[script]+args
    subprocess.run(cmd)
    return

In [6]:
input_dir = 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/'
input_files = glob(input_dir+'*.mnc')
mask = 'data/human/registration/reference_files/mask_resampled_3.0.mnc'

In [17]:
outfile = None

df_imgs = import_images(infiles = input_files,
                                   mask = mask,
                                   output_format = 'pandas',
                                   flatten = True,
                                   parallel = True,
                                   nproc = 2)

df_imgs['file'] = input_files
if outfile is None:
    outfile = 'effect_sizes.csv'

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 126.17it/s]


array(['data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050158_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
       'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050100_ses-01_run-02_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
       'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050098_ses-01_run-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
       'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050097_ses-01_run-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
       'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050101_ses-01_run-02_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
       'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshol

In [27]:
def build_voxel_matrix(infiles, mask = None, save = False, outfile = 'voxel_matrix.csv', parallel = False, nproc = None):
    
    """
    Create a matrix of voxels from a set of images.
    
    Arguments
    ---------
    infiles: list
        List of paths to images.
    mask: str
        Optional path to a mask image. 
    save: bool
        Option to save to CSV.
    outfile: str
        Path to the output CSV file. Ignored if save = False.
    parallel: bool
        Option to run in parallel.
    nproc: int
        Number of processors to use in parallel.
    
    Returns
    -------
    df_imgs
        A Pandas DataFrame containing the (masked) images.
    """
    
    df_imgs = import_images(infiles = infiles,
                           mask = mask,
                           output_format = 'pandas',
                           flatten = True,
                           parallel = parallel,
                           nproc = nproc)
    df_imgs['file'] = infiles
    
    if save:
        df_imgs.to_csv(outfile, index = False)
    
    return df_imgs

In [28]:
output_file = 'data/human/effect_sizes/absolute/human_effect_sizes_absolute_3.0mm.csv'
build_voxel_matrix(infiles = input_files,
                  mask = mask,
                   save = True,
                   outfile = output_file,
                  parallel = False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 64.20it/s]


,0,1,2,3,4,5,6,7,8,9,...,53399,53400,53401,53402,53403,53404,53405,53406,53407,file
0,0.039293,-0.599063,-0.377588,-0.788395,-0.403643,-0.493537,-0.491797,0.359342,0.109210,0.104862,...,1.347266,0.644209,0.395816,0.039717,0.909528,0.929512,0.282028,0.688070,-0.141356,data/human/effect_sizes/absolute/resolution_3....
1,-0.550862,-0.351340,-0.636478,-0.381057,-0.400159,-0.213358,-0.141545,0.129444,-0.427039,-0.283058,...,1.537093,0.911292,1.037576,-1.883201,1.662702,0.707169,0.279815,-0.642503,-1.145560,data/human/effect_sizes/absolute/resolution_3....
2,0.349136,0.222828,0.305279,0.028101,0.003543,0.823964,0.425153,0.594155,0.574852,0.724551,...,-0.682405,-1.074788,-1.048437,-0.342072,-1.284671,-1.153134,-0.587053,-1.526175,-0.342615,data/human/effect_sizes/absolute/resolution_3....
3,0.248457,0.278657,0.173421,0.286206,0.339533,0.281958,0.213533,0.528771,0.379644,0.544809,...,-1.218244,-0.890247,-0.712806,0.594351,-1.291379,-1.096483,0.089887,-0.633554,0.647225,data/human/effect_sizes/absolute/resolution_3....
4,-1.605346,-1.708406,-1.784165,-2.060301,-2.200636,-0.402716,-0.747552,-0.826623,-0.636613,-0.728925,...,0.554403,0.156562,-0.319093,-0.378303,0.477017,0.606179,0.012920,0.646741,0.047295,data/human/effect_sizes/absolute/resolution_3....
5,1.304991,1.300784,1.502293,1.723310,1.426885,1.251940,1.485532,1.476679,1.128628,1.438059,...,0.705651,0.484607,-0.304579,1.367329,2.495751,1.322206,0.378507,2.304020,0.720991,data/human/effect_sizes/absolute/resolution_3....
6,1.850592,1.825994,1.549437,1.646103,1.693011,1.110983,1.465058,1.331495,0.745751,0.905070,...,0.926227,0.105105,-0.189767,-0.153433,0.752616,0.487210,0.508098,0.207499,0.648227,data/human/effect_sizes/absolute/resolution_3....
7,-0.932474,-1.587430,-0.557165,-1.202698,-0.556656,-1.355891,-1.348442,-1.724766,-0.207126,-0.701943,...,0.667458,1.240109,2.151893,-0.141185,0.103745,0.614438,1.859378,0.825129,2.800390,data/human/effect_sizes/absolute/resolution_3....
8,0.735738,0.954979,0.814499,0.879412,1.031081,1.393460,1.759050,1.380174,0.879937,1.034276,...,1.319521,0.938491,1.712218,1.599415,1.698400,1.123665,1.951679,2.389111,2.154973,data/human/effect_sizes/absolute/resolution_3....
9,-0.322005,-0.352417,0.088579,-0.196257,-0.500389,0.180990,0.155249,-0.314401,-0.082199,0.045301,...,-0.673505,-1.288779,-1.127950,-0.467649,-1.127950,-1.987165,-1.980114,-1.753212,-2.203544,data/human/effect_sizes/absolute/resolution_3....
